In [1]:
import pandas as pd
import numpy as np
import glob
import json

In [2]:
#Step-1: Get filenames from the directory for setting up the dropdown.
FullFilenames = (glob.glob("Dataset/*.csv"))
OnlyFilenames = []
for i in FullFilenames:
    SplitOnUnderscores = i.split('_')
    SplitOnSlash = SplitOnUnderscores[0].split('\\')
    filename = SplitOnSlash[1]
    OnlyFilenames.append(filename)

In [3]:
#Step-2: Create JSON for city dropdown using extracted filenames
cities= json.dumps({"cities": OnlyFilenames})
f = open("city_dropdown.json", "w")
f.write(cities)
f.close()

In [21]:
#Step-3: Clean the dataset of given cities and save them as unified json data with trees count grouped by city
# For the sake of data integrity, We will be dropping the rows which have null values
City = ['LosAngeles'] # set the city names you wish to get data for
IntrestingColumns = ['common_name','scientific_name','city','state'] # insert the names of columns you wish to extract
tree_data_by_city = {}

for city in City:
    Filename = (glob.glob("Dataset/" + city + "*.csv"))
    data = pd.read_csv(Filename[0], usecols=InterestingColumns) #use nrows paramter to limit the data if it consumes too much time to process
    data.dropna(how='any', inplace=True)

    # Group data by scientific_name and get the count
    tree_counts = data['scientific_name'].value_counts().to_dict()

    # Extract city and state information (assuming they are the same for all records in the city)
    city_info = data.iloc[0][['city', 'state']].to_dict()

    # Create a list of dictionaries for each tree species in the city
    city_tree_data = []
    for scientific_name, count in tree_counts.items():
        print(scientific_name)
        common_names = data[data['scientific_name'] == scientific_name]['common_name'].unique().tolist()
        
        city_tree_data.append({
            'scientific_name': scientific_name,
            'common_name': common_names,
            'count': count,
            'state': city_info['state']
        })

    # Store the city-specific tree data in the dictionary
    tree_data_by_city[city] = city_tree_data


tree_data_json = json.dumps(tree_data_by_city)
with open("tree_data.json", "w") as f:
    f.write(tree_data_json)


Washingtonia robusta
Lagerstroemia indica
Magnolia grandiflora
Liquidambar styraciflua
Ficus microcarpa
Platanus acerifolia
Cinnamomum camphora
Cupressus sempervirens
Ulmus parvifolia
Jacaranda acutifolia
Cupaniopsis anacardioides
Fraxinus velutina
Pinus canariensis
Arecastrum romanzoffianum
Ceratonia siliqua
Pyrus kawakamii
Phoenix canariensis
Quercus agrifolia
Pittosporum undulatum
Pyrus calleryana
Fraxinus uhdei
Schinus terebinthifolius
Ficus benjamina
Jacaranda mimosifolia
Ligustrum lucidum
Callistemon lanceolatus
Morus alba
Ulmus pumila
Brachychiton populneus
Callistemon citrinus
Platanus racemosa
Pinus halepensis
Podocarpus gracilior
Podocarpus macrophyllus
Prunus caroliniana
Juglans californica
Quercus
Nerium oleander
Schinus molle
Cedrus deodara
Eucalyptus globulus
Betula pendula
Juniperus
Olea europaea
Liriodendron tulipifera
Tristania conferta
Washingtonia filifera
Eucalyptus rudis
Acacia melanoxylon
Eucalyptus
Yucca gloriosa
Pinus thunbergiana
Robinia pseudoacacia
Quercus il